In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Bootcamp/pjt/train_model

/content/drive/MyDrive/Bootcamp/pjt/train_model


In [3]:
!git pull origin main

remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 1), reused 3 (delta 1), pack-reused 0
Unpacking objects: 100% (3/3), 253 bytes | 5.00 KiB/s, done.
From https://github.com/hoon0528/pjt_ai_speaker
 * branch            main       -> FETCH_HEAD
   fd256b1..9120734  main       -> origin/main
Updating fd256b1..9120734
Fast-forward
 .gitignore | 1 +
 1 file changed, 1 insertion(+)
 create mode 100644 .gitignore


In [4]:
!unzip -j /content/drive/MyDrive/Bootcamp/pjt/loopy_mono.zip -d /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/wavs

Archive:  /content/drive/MyDrive/Bootcamp/pjt/loopy_mono.zip
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (1).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (10).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (100).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (101).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (102).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (103).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (104).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (105).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (106).wav  
  inflating: /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/loopy (107).wav  
  inflating: /content/drive/MyDrive/Bootca

In [3]:
# 2번 실행 (런타임 초기화 후)
!pip install TTS

In [3]:
# 2번 실행 (런타임 초기화 후)
!pip install --upgrade pip
!pip install --upgrade tensorflow
!pip install --upgrade tensorboard

In [4]:
import os

from pathlib import Path
from trainer import Trainer, TrainerArgs

from TTS.tts.configs.shared_configs import BaseDatasetConfig, CharactersConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import Vits, VitsArgs, VitsAudioConfig
from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [5]:
# 본인 데이터 경로에 맞게 수정
dir_data_root = Path("/content/drive/MyDrive/Bootcamp/pjt/train_model/data")
character = 'loopy'

dataset_config = BaseDatasetConfig(formatter="ljspeech", meta_file_train="transcriptions.txt", path=dir_data_root/character)

In [6]:
audio_config = VitsAudioConfig(
    sample_rate=44100, win_length=1024, hop_length=256, num_mels=80, mel_fmin=0, mel_fmax=None
)

In [7]:
# 1. 발음에 사용된 문자 있을 시 추가 (&: 앤, @: 앳 등)
# 2. punctuation에 사용한 글자 외의 온점 반점 등 전부 추가
korean_characters = ''.join([chr(i) for i in range(ord('가'), ord('힣')+1)])
english_characters = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz1234567890&" # 수정 1.

character_config = CharactersConfig(
    characters_class= "TTS.tts.models.vits.VitsCharacters",
    characters= korean_characters + english_characters,
    punctuations=" ?!,.~", # 수정 2.
    pad= "<PAD>",
    eos= "<EOS>",
    bos= "<BOS>",
    blank= "<BLNK>",
)

In [8]:
config = VitsConfig(
    audio=audio_config,
    characters=character_config,
    run_name="vits_vctk",
    batch_size=50,
    eval_batch_size=4,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=0,
    epochs=1000,
    text_cleaner="basic_cleaners",
    use_phonemes=False,
    phoneme_language="ko-kr",
    phoneme_cache_path=dir_data_root/character,
    compute_input_seq_cache=True,
    print_step=25,
    print_eval=False,
    save_best_after=1000,
    save_checkpoints=True,
    save_all_best=True,
    mixed_precision=True,
    max_text_len=250,  # change this if you have a larger VRAM than 16GB
    output_path=dir_data_root,
    datasets=[dataset_config],
    cudnn_benchmark=False,
    test_sentences=[
        ["이것은 테스트 음성입니다."],
        ["모델을 훈련할 때 주의할 몇 가지 사항이 있습니다."],
        ["좋은 아침입니다~!"]
    ]
)

In [9]:
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# config is updated with the default characters if not defined in the config.
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:44100
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:0
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:None
 | > fft_size:1024
 | > power:None
 | > preemphasis:0.0
 | > griffin_lim_iters:None
 | > signal_norm:None
 | > symmetric_norm:None
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:None
 | > pitch_fmax:None
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:1.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [10]:
# 1. wav_file 경로 수정
# 추가적으로 수정할 사항 있을 시 수정

def formatter(root_path, manifest_file, **kwargs):  # pylint: disable=unused-argument
    """Assumes each line as ```<filename>|<transcription>```
    """
    txt_file = os.path.join(root_path, manifest_file)
    items = []
    speaker_name = "my_speaker"
    with open(txt_file, "r", encoding="utf-8") as ttf:
        for line in ttf:
            cols = line.split("|")
            wav_file = f"/content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy/wavs/{cols[0]}.wav" # 본인 폴더에 맞게 수정
            text = cols[1]
            # print(text)
            items.append({"text":text, "audio_file":wav_file, "speaker_name":speaker_name, "root_path": root_path})
    return items

In [11]:
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=True, formatter=formatter)

 | > Found 400 files in /content/drive/MyDrive/Bootcamp/pjt/train_model/data/loopy


In [12]:
# init model
model = Vits(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(),
    config,
    dir_data_root,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/Bootcamp/pjt/train_model/data/vits_vctk-July-17-2024_05+50AM-2009e53

 > Model has 85192684 parameters


In [ ]:
trainer.fit()


 > EPOCH: 0/1000
 --> /content/drive/MyDrive/Bootcamp/pjt/train_model/data/vits_vctk-July-17-2024_05+50AM-2009e53

 > TRAINING (2024-07-17 05:51:00) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: True
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 396
 | > Preprocessing samples
 | > Max text length: 89
 | > Min text length: 8
 | > Avg text length: 41.54040404040404
 | 
 | > Max audio length: 522892.0
 | > Min audio length: 64555.0
 | > Avg audio length: 317665.3080808081
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
